In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv("../data/Train.csv")
#test=pd.read_csv("./data/Test.csv")
meta=pd.read_csv("../data/airqo_metadata.csv")

In [3]:
train.head()


,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [4]:
# covert features  from string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
        
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    #test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

count nan values

In [5]:
list_columns = ['temp', 'precip', 'rel_humidity', 'wind_dir', 'wind_spd', 'atmos_press']
def count_nan(row):
    counter = 0
    if isinstance(row, type(list())):
        for x in row:
            if np.isnan(x):
                counter += 1
        return counter 
    else: return np.nan
    
for col in list_columns:
    train['nan_' + str(col)] = round(train[col].apply(count_nan))
data_nan = train.drop(list_columns, axis=1)

data_nan['sum'] = round(data_nan.sum(axis=1, numeric_only=True)/726*100,2)
data_nan.shape

(15539, 10)

In [6]:
data_wo_outlier = train.join(data_nan['sum'])[(train.join(data_nan['sum'])['sum']<50)]
data_wo_outlier.shape

(14536, 16)

In [7]:
data_wo_outlier.head(2)

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target,nan_temp,nan_precip,nan_rel_humidity,nan_wind_dir,nan_wind_spd,nan_atmos_press,sum
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702,0,0,0,0,0,0,10.90
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304,1,1,1,46,46,1,17.72


fill nan values with mean

In [8]:
# removing the nan values and show the remain values
def mean_without_nan(row):
    mean = np.nanmean(row)
    new_row = []
    for x in row: 
        if np.isnan(x): new_row.append(mean)
        else: new_row.append(x)
    return new_row

In [9]:
features = data_wo_outlier.columns[2:8]
features
nan_featuers = ["nan_"+ str(feature) for feature in features]
nan_featuers

['nan_temp',
 'nan_precip',
 'nan_rel_humidity',
 'nan_wind_dir',
 'nan_wind_spd',
 'nan_atmos_press']

In [10]:
# fill nan
for feat in features:
    data_wo_outlier["new_" + str(feat)] = data_wo_outlier[feat].apply(mean_without_nan)




In [11]:
features_to_drop = features #.extend(nan_featuers,['sum'])
data_process = data_wo_outlier.drop(features, axis=1)
data_process.drop(nan_featuers, axis=1, inplace=True)
data_process.drop(['sum'], axis=1, inplace=True)


In [12]:
def convert_vector(row):
    new_row = []
    rad = []
    for x in row: 
        rad.append(x*np.pi/180)
    
    for r in rad: 
        wx = round(np.cos(r),2)
        wy = round(np.sin(r),2)
        new_row.append([wx,wy])
    return new_row
    

data_process['new_wind_vec'] = data_process['new_wind_dir'].apply(convert_vector)

In [13]:
features = [x for x in features if x != 'wind_dir']
features.append('wind_vec')

In [14]:
data_process.drop(['new_wind_dir'], axis=1, inplace=True)
data_process.head()

,ID,location,target,new_temp,new_precip,new_rel_humidity,new_wind_spd,new_atmos_press,new_wind_vec
1,ID_train_1,D,79.131702,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...","[[0.2, -0.98], [0.01, 1.0], [0.14, 0.99], [0.3..."
2,ID_train_10,A,32.661304,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[1.0386555556, 1.0386555556, 1.0386555556, 1.0...","[88.55166667, 88.46416667, 88.31916667, 88.24,...","[[-0.93, -0.36], [-0.93, -0.36], [-0.93, -0.36..."
3,ID_train_100,A,53.850238,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...","[[0.5, -0.87], [0.4, -0.92], [0.41, -0.91], [0..."
5,ID_train_10000,E,17.005000,"[26.225, 26.25, 26.95833333, 27.925, 28.416666...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7566666670000001, 0.7090833329999999, 0.632...","[1.1332211538846155, 1.4858333330000002, 1.090...","[88.56583333, 88.55, 88.50916667, 88.44083333,...","[[-0.98, 0.2], [-0.25, 0.97], [-0.95, 0.32], [..."
6,ID_train_10001,C,53.100000,"[21.86666667, 21.38333333, 20.75, 20.74166667,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.548333333, 0.571666667, 0.589166667, 0.57, ...","[1.1525, 1.055, 1.11, 1.385833333, 1.2, 1.2508...","[87.73666667, 87.74416667, 87.7475, 87.78, 87....","[[0.9, -0.44], [0.89, -0.46], [0.85, -0.52], [..."


- create new dataframes for each feature and observation

In [15]:
# name of dataframes:
df_names = ["df_" + str(feat) for feat in features]
df_names

['df_temp',
 'df_precip',
 'df_rel_humidity',
 'df_wind_spd',
 'df_atmos_press',
 'df_wind_vec']

In [16]:
features
for f in features: 
    exec(f'df_{f} = pd.DataFrame(data_process["new_{f}"])')
    

unpack all values: 

In [17]:
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]
    for x in range(121): # 121 messdaten pro punkt 
        df[name+ str(x)] = df[name].str[x]
        #df.drop(name, axis=1, inplace=True)
    

/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_2634/4261563240.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name+ str(x)] = df[name].str[x]
/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_2634/4261563240.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name+ str(x)] = df[name].str[x]
/var/folders/gz/t1kyprv95tzc2bf6b375z94h0000gn/T/ipykernel_2634/4261563240.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [18]:
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]
    df.drop(name, axis=1, inplace=True)

add meta information

In [19]:
meta.head()

,Unnamed: 0,location,loc_altitude,km2,aspect,dist_motorway,dist_trunk,dist_primary,dist_secondary,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,0,A,1122.4,1.9,194.0,NaN,NaN,14.695789,343.595039,575.917422,330.609776,254.307415,4763,809,508,43,142
1,1,B,1155.4,5.4,219.8,NaN,528.078476,2172.680462,1144.376412,531.103271,65.142004,1.042809,22243,5735,116,1144,239
2,2,C,1178.3,8.5,168.7,NaN,32.885520,4794.704552,12.730489,72.459340,406.967815,13.931350,97895,26873,21316,751,9835
3,3,D,980.8,0.8,90.0,NaN,NaN,265.896472,NaN,339.693908,76.024389,11.601442,3596,663,7,651,99
4,4,E,1186.5,1.6,121.0,NaN,850.423131,NaN,1738.872942,137.722087,449.591885,3.836589,6064,1297,985,26,43


input nan dist_trunk, dist_primary and dist_secondary with knn imputer

In [20]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
meta_imp = imputer.fit_transform(meta.drop(['location'], axis=1))

In [21]:
meta_imp = pd.DataFrame(meta_imp)
meta_imp.columns = [col for col in meta.columns[1:] if col != 'dist_motorway']
meta_imp

,location,loc_altitude,km2,aspect,dist_trunk,dist_primary,dist_secondary,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,0.0,1122.4,1.9,194.0,689.250804,14.695789,343.595039,575.917422,330.609776,254.307415,4763.0,809.0,508.0,43.0,142.0
1,1.0,1155.4,5.4,219.8,528.078476,2172.680462,1144.376412,531.103271,65.142004,1.042809,22243.0,5735.0,116.0,1144.0,239.0
2,2.0,1178.3,8.5,168.7,32.885520,4794.704552,12.730489,72.459340,406.967815,13.931350,97895.0,26873.0,21316.0,751.0,9835.0
3,3.0,980.8,0.8,90.0,689.250804,265.896472,1041.233990,339.693908,76.024389,11.601442,3596.0,663.0,7.0,651.0,99.0
4,4.0,1186.5,1.6,121.0,850.423131,140.296131,1738.872942,137.722087,449.591885,3.836589,6064.0,1297.0,985.0,26.0,43.0


In [22]:
meta_imp['location'] = meta.location

In [23]:
meta_imp

,location,loc_altitude,km2,aspect,dist_trunk,dist_primary,dist_secondary,dist_tertiary,dist_unclassified,dist_residential,popn,hh,hh_cook_charcoal,hh_cook_firewood,hh_burn_waste
0,A,1122.4,1.9,194.0,689.250804,14.695789,343.595039,575.917422,330.609776,254.307415,4763.0,809.0,508.0,43.0,142.0
1,B,1155.4,5.4,219.8,528.078476,2172.680462,1144.376412,531.103271,65.142004,1.042809,22243.0,5735.0,116.0,1144.0,239.0
2,C,1178.3,8.5,168.7,32.885520,4794.704552,12.730489,72.459340,406.967815,13.931350,97895.0,26873.0,21316.0,751.0,9835.0
3,D,980.8,0.8,90.0,689.250804,265.896472,1041.233990,339.693908,76.024389,11.601442,3596.0,663.0,7.0,651.0,99.0
4,E,1186.5,1.6,121.0,850.423131,140.296131,1738.872942,137.722087,449.591885,3.836589,6064.0,1297.0,985.0,26.0,43.0


add the meta data into each dataframe according to the location

- add meta daten to the df


In [24]:
for dframe in df_names:
    df = locals()[dframe]
    
    name = df.columns[0]
    df_1 = df.join(data_process["location"])
    exec(f'f_{name} = pd.merge(df_1, meta_imp, how="left", on="location")')

In [25]:
list_features = []
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]
    list_features.append("f_"+str(name))

In [26]:
list_features

['f_new_temp0',
 'f_new_precip0',
 'f_new_rel_humidity0',
 'f_new_wind_spd0',
 'f_new_atmos_press0',
 'f_new_wind_vec0']

In [27]:
target = data_process.target

In [28]:
list_features = []
for dframe in df_names:
    df = locals()[dframe]
    print(df.shape)

(14536, 121)
(14536, 121)
(14536, 121)
(14536, 121)
(14536, 121)
(14536, 121)


In [29]:
data_process.shape

(14536, 9)

In [30]:
# export data



## Model forecast for each feature

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [32]:
def train_predict(model, df, target):
    # data split
    X_train, X_test, y_train, y_test = train_test_split(df, target,random_state=44)
    #model
    
    model.fit(X_train, y_train)
    # predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    rsme_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rsme_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

    test_results[str(model) + "_"+ df.columns[0]] = [rsme_train, rsme_test]

    print(df.columns[0])
    print (f'[{rsme_train:.2f}, {rsme_test:.2f}]')
    return y_pred_train



In [33]:
df_names

['df_temp',
 'df_precip',
 'df_rel_humidity',
 'df_wind_spd',
 'df_atmos_press',
 'df_wind_vec']

In [34]:
df_temp

,new_temp0,new_temp1,new_temp2,new_temp3,new_temp4,new_temp5,new_temp6,new_temp7,new_temp8,new_temp9,...,new_temp111,new_temp112,new_temp113,new_temp114,new_temp115,new_temp116,new_temp117,new_temp118,new_temp119,new_temp120
1,22.533333,21.716667,20.833333,20.983333,20.875000,20.141667,19.375000,18.791667,18.775000,18.641667,...,25.816667,26.791667,27.833333,29.600000,30.233333,30.583333,28.466667,26.991667,26.025000,21.450000
2,28.975000,27.950000,29.600000,26.425000,22.091667,21.775000,22.333333,21.950000,20.441667,20.441667,...,21.816667,20.733333,20.508333,20.441667,20.991667,23.541667,26.408333,28.075000,29.241667,30.091667
3,22.966667,24.266667,25.275000,25.625000,25.866667,25.091667,24.025000,22.425000,20.233333,17.416667,...,19.241667,18.291667,18.075000,17.741667,17.625000,18.308333,20.325000,21.900000,23.533333,24.641667
5,26.225000,26.250000,26.958333,27.925000,28.416667,27.850000,27.225000,26.466667,25.591667,24.141667,...,19.766667,19.583333,19.375000,19.883333,19.575000,19.833333,20.375000,21.625000,23.466667,25.075000
6,21.866667,21.383333,20.750000,20.741667,20.583333,20.966667,21.300000,22.350000,22.925000,23.600000,...,25.533333,24.741667,23.991667,23.491667,22.991667,23.250000,22.841667,19.391667,19.116667,19.508333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15534,22.716667,16.933333,18.033333,18.975000,19.383333,19.600000,19.566667,19.500000,19.158333,19.158333,...,19.300000,19.316667,19.291667,19.000000,18.900000,19.791667,21.975000,23.875000,24.991667,24.991667
15535,25.375000,26.025000,26.400000,26.233333,25.700000,24.900000,23.208333,22.583333,22.316667,22.108333,...,20.708333,20.250000,20.158333,20.875000,24.241667,25.708333,26.666667,27.700000,27.900000,27.925000
15536,26.091667,28.975000,30.316667,30.966667,30.033333,28.300000,26.391667,24.433333,23.066667,21.841667,...,20.475000,20.508333,20.700000,21.841667,22.433333,22.708333,24.175000,23.258333,25.566667,28.458333
15537,29.225000,30.000000,29.550000,29.666667,27.725000,24.466667,22.775000,21.800000,19.625000,19.000000,...,16.858333,16.408333,17.741667,21.750000,24.225000,25.927273,27.258333,28.483333,29.454545,30.308333


In [44]:
for dframe in df_names: 
    df = locals()[dframe]
    name = df.columns[0][4:-1]
    print(name)
    df.to_csv('./data/'+ str(name)+'.csv') 
    del df
target.to_csv('./data/target.csv')

temp
precip
rel_humidity
wind_spd
atmos_press
wind_vec


In [ ]:
test_results = {}
linear = LinearRegression()
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]

    train_predict(linear, df, target)


In [ ]:
pd.DataFrame(test_results).T

temperatur has the best fit


# dnn

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
def model_compile_and_fit(df, target, max_epochs=30):
    # Get optimizer
    #optimizer=tf.keras.optimizers.Adam()
    # data split
    X_train, X_test, y_train, y_test = train_test_split(df, target,random_state=44)

    # model 
    with tf.device('/cpu:0'):
      model = tf.keras.Sequential([
            tf.keras.layers.Dense(32,kernel_initializer = 'uniform', activation='relu', input_dim = 121),
            #tf.keras.layers.Dense(121,kernel_initializer = 'uniform', activation='relu', input_dim = 121),
            tf.keras.layers.Dense(1,kernel_initializer = 'uniform')
            ])

    N_VAL =  len(X_test)
    N_TRAIN = len(X_train)
    BATCH_SIZE = 96
    STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
    EPOCHS = 500

    # model.compile
    model.compile(optimizer='Adam',
                metrics='mse', # [tf.keras.metrics.RootMeanSquaredError()]
                loss='mae')
    # model.fit
    with tf.device('/cpu:0'):
      train_history = model.fit(X_train, 
                        y_train,
                        validation_split=0.2,
                        verbose=1,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS, 
                        )
   

    # Predict values for test set
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

    print(df.columns[0])
    print (f'[{rmse_train:.2f}, {rmse_test:.2f}]')

    test_results[str(model) + "_"+ df.columns[0]] = [rmse_train, rmse_test]
    pass

In [ ]:
for dframe in df_names:
    df = locals()[dframe]
    name = df.columns[0]

    model_compile_and_fit(df, target)

In [ ]:
pd.DataFrame(test_results).T